In [1]:
import sys
sys.path.append('..')
import random

from tqdm import tqdm
import numpy as np
from langchain.chat_models import ChatVertexAI
import chromadb
from finetune.auto_label import *

SEED = 93
np.random.seed(SEED)
random.seed(SEED)
tqdm.pandas()

In [2]:
TUNED = "flan"
FINETUNED_PATH = "../data/summary_eval_shilong.xlsx"
OG_PATH = "../data/summary_eval_og_shilong.xlsx"
TEST_SET_PATH = f"../data/{TUNED}-t5-xxl-test-predicted.parquet"
TEST_SET_RATED_PATH = f"../data/{TUNED}-t5-xxl-rated.parquet"
TEMPERATURE = 0
MAX_OUTPUT = 1024
MODEL_NAME = "chat-bison"

In [3]:
finetuned_sample = pd.read_excel(FINETUNED_PATH)
og_sample = pd.read_excel(OG_PATH)
combined_sample = pd.concat([finetuned_sample, og_sample], ignore_index=True).reset_index(drop=True)
combined_sample.head()

,source,id,category,title,published,body,summary,summary_type,predicted,rating,thought
0,reuters,22071,Software company morphs into personal injury firm,Plaintiffs' BarLegal InnovationLegal IndustryL...,2022-06-27T22:46:00,summarize in bullet points: (Reuters) - A soft...,* Mighty Law says it will undercut personal in...,BULLETS,\n* A software company is opening its own law ...,4,The summary accurately captures the main point...
1,cnbc,8940,Personal Finance,College hopefuls have a new ultimate dream sch...,2023-03-15T13:11:19+00:00,summarize in bullet points: ## Massachusetts I...,"* This year, the school named by the highest n...",BULLETS,\n* A new survey of college-bound students and...,3,The summary accurately captures the main point...
2,reuters,50327,LNGFuel OilGasHydrogenRenewable Fuels,Spain's Cepsa to invest 3 bln euros in green h...,2022-12-01T15:03:00,"summarize in bullet points: CADIZ, Spain, Dec ...","* To produce 300,000 tons of green hydrogen pe...",BULLETS,\n* Cepsa to invest 3 bln euros in renewable e...,3,Thought process for the rating:\n- The summary...
3,reuters,22786,U.S. Markets,"Wall St falls after recent strong gains, Alpha...",2023-02-09T00:30:00,"summarize in bullet points: NEW YORK, Feb 8 (R...",* Alphabet shares dive after Google AI chatbot...,BULLETS,"\n* S&P 500 futures down 1.2%, yuan up 0.2%, w...",2,Thought process for the rating:\n- The summary...
4,cnbc,1864,Sports,Canadian billionaire Steve Apostolopoulos says...,2023-04-24T17:42:10+00:00,summarize in bullet points: Canadian billionai...,* Canadian businessman Steve Apostolopoulos sa...,BULLETS,\n* The saga over Washington's embattled footb...,4,The summary reflects the content of the text w...


In [4]:
len(combined_sample.index)

20

In [5]:
chroma = chromadb.EphemeralClient()
plan_llm = ChatVertexAI(
    temperature=TEMPERATURE,
    model_name=MODEL_NAME,
    max_output_tokens=MAX_OUTPUT,
)
format_examples = [{
    "rating": config.LABEL_FORMAT_RATING,
    "raw": config.LABEL_FORMAT_RAW,
    "thought": config.LABEL_FORMAT_THOUGHT
}]
format_selector = RandomExampleSelector(examples=format_examples, k=1)

def rate_summary(row, selector):
    text = RatableText(context_text=row["body"], output=row["predicted"])
    try:
        result = evaluate_text(plan_llm, text, config.LABEL_SUMMARY_SYSTEM, config.LABEL_SUMMARY_USER, meal_selector=selector, format_selector=format_selector)
    except ValueError as e:
        result = RatingOutput(rating=np.nan, thought=str(e)).model_dump()
    return result

In [6]:
sample_ratings = []
for idx, row in tqdm(combined_sample.iterrows(), total=len(combined_sample.index)):
    subsample = combined_sample.drop(idx)
    collection = chroma.get_or_create_collection("temp-examples")
    collection.query(query_texts=["warm-up"], n_results=1)
    records = create_db_entries(subsample, collection)
    collection.add(**records)
    assert len(records["ids"]) == collection.count()
    meal_selector = ChromaRatingExampleSelector(collection)
    sample_ratings.append(
        rate_summary(row, meal_selector)["rating"]
    )
    chroma.delete_collection("temp-examples")
combined_sample["predicted_rating"] = sample_ratings
combined_sample[["rating", "predicted_rating"]].head()

100%|██████████| 20/20 [02:31<00:00,  7.57s/it]


,rating,predicted_rating
0,4,4.0
1,3,3.0
2,3,3.0
3,2,1.0
4,4,4.0


In [7]:
combined_sample["rating_diff"] = combined_sample.rating - combined_sample.predicted_rating
print(np.mean(combined_sample.rating_diff.abs()))
print(combined_sample.rating_diff.describe())

1.0
count    18.000000
mean     -0.111111
std       1.450715
min      -4.000000
25%      -1.000000
50%       0.000000
75%       1.000000
max       2.000000
Name: rating_diff, dtype: float64


In [8]:
test_set = pd.read_parquet(TEST_SET_PATH)
test_set.head()

,source,id,category,title,published,body,summary,summary_type,predicted
0,reuters,112266,China,Millions tested in Shanghai as China grapples ...,2022-07-07T21:01:00,"summarize in bullet points: SHANGHAI/BEIJING, ...",* Third day of mass testing in many Shanghai d...,BULLETS,"China's most populous city, Shanghai, has just..."
1,reuters,66543,Business,Ecigarettes and heated tobacco light up Imperi...,2022-05-17T08:37:00,"summarize in bullet points: LONDON, May 17 (Re...",* H1 adjusted net revenue up 0.3% at constant ...,BULLETS,Imperial Brands is on track to meet its full-y...
2,cnbc,7668,Sustainable Future,Private jet flights in Europe soar to record l...,2023-03-30T05:24:48+00:00,summarize in bullet points: A private jet is l...,* Analysis published Thursday by environmental...,BULLETS,Private jet flights in Europe last year rose b...
3,reuters,87382,Middle East,Israel's Netanyahu returns with hard-right cab...,2022-12-29T19:28:00,"summarize in bullet points: JERUSALEM, Dec 29 ...","* New government has nationalist, religious pa...",BULLETS,Benjamin Netanyahu was sworn in as Israel's pr...
4,reuters,15571,DiversityCorporate CounselCorporate Governance,Activist behind Harvard race case takes aim at...,2021-07-13T21:56:00,summarize in bullet points: (Reuters) - Edward...,* Lawsuit claims statutes discriminate against...,BULLETS,"Edward Blum, the affirmative action opponent b..."


In [ ]:
collection = chroma.get_or_create_collection("temp-examples")
collection.query(query_texts=["warm-up"], n_results=1)
records = create_db_entries(combined_sample, collection)
collection.add(**records)
assert len(records["ids"]) == collection.count()
meal_selector = ChromaRatingExampleSelector(collection)

ratings = test_set.progress_apply(lambda row: rate_summary(row, meal_selector), axis=1).tolist()
ratings[0]

 61%|██████▏   | 2779/4527 [5:02:32<4:32:31,  9.35s/it] 

In [ ]:
test_set["rating"] = [r["rating"] for r in ratings]
print(test_set.rating.describe())

In [ ]:
test_set.to_parquet(TEST_SET_RATED_PATH, index=False)